In [2]:
import torch 
from torch import nn
from torch import Tensor

device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)



cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class My_CBOW(nn.Module):
    def __init__(self, vob_size:int, embed_dim:int, window_size:int):
        super(My_CBOW,self).__init__()
        self.inEmbedding = nn.Embedding(vob_size,embed_dim)
      
        size = embed_dim * window_size * 2;

#        print(f"size =={size}")
        # self.linear1 = nn.Linear(size, embed_dim)

        self.outEmbedding = nn.Embedding(vob_size,embed_dim)

    #    self.h = None
        
    
    def makeH(self,input):
        W = self.inEmbedding(input)
        n = W.shape[-2]


  #      print(f"W.shape = {W.shape}")
        h = torch.sum(W,dim=-2) / n
    #     x = W.view(input.shape[0],-1)


    #  #   print(f"x.shape = {x.shape}")
    #     h = self.linear1(x)
    #     h = nn.functional.relu(h)

        return h

    def forwardWithH(self,h,out):
        W = self.outEmbedding(out)
        score = torch.sum(h * W,dim = -1)
        return score



# ## 模型测试
# cbow = My_CBOW(10,3,3)
# cbow.to(device=device)
# input = torch.Tensor([[1,2,5],[1,2,3],[6,2,3],[6,2,3]]).to(dtype=torch.int32,device = device)

# ##正样本
# h = cbow.makeH(input)

# print(f"shape{h.shape}")
# pout = Tensor([1,3,2,5]).to(device=device,dtype=int)
# plable = torch.ones(pout.shape).to(device = device)
# pscore = cbow.forwardWithH(h,pout).to(device = device)


# ##负样本
# nout = Tensor([1,0,1,7]).to(device=device,dtype=int)
# nlable = torch.zeros_like(nout).to(device = device)
# nsocre = cbow.forwardWithH(h,nout).to(device = device)


# score = torch.cat((pscore,nsocre))
# lables = torch.cat((plable,nlable)).to(device = device)
# print(score)



# loss_fn = torch.nn.BCEWithLogitsLoss()
# loss = loss_fn(score,lables)

# print(loss)

# loss.backward()

# input = Tensor([[1,2,5],[1,2,3],[6,2,3]],dtype=torch.int)
# print(input)

# cbow.makeH(input)



In [ ]:
import sys

sys.path.append("/content/drive/MyDrive/ml/nlp/")

import numpy
import pickle


from common.util import create_contexts_target
from dataset import ptb


window_size = 5
hidden_size =  100

batch_size = 100
max_epoch = 10


corpus, word_to_id, id_to_word = ptb.load_data("train")
vocab_size = len(word_to_id)

context , target = create_contexts_target(corpus=corpus,window_size=window_size)

print(context.shape)
print(target)



(929579, 10)
[   5    6    7 ...  552  917 3196]


In [ ]:
import collections
import numpy as np
GPU = device == "cuda"
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)
    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # 在用GPU(cupy）计算时，优先速度
            # 有时目标词存在于负例中
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [ ]:
def train(model:My_CBOW, loss_fn , optimizer:torch.optim.Optimizer ,context : Tensor, target: Tensor, negativeTarget: Tensor):

    model.zero_grad()
    # model.train()
    h = model.makeH(context)

    p_lable = torch.ones_like(target)
    

    p_score = model.forwardWithH(h,target)

    n_lable = torch.zeros_like(negativeTarget)
    n_score = model.forwardWithH(h,negativeTarget)

    score = torch.cat((p_score,n_score)).to(dtype=torch.float32)
    lables = torch.cat((p_lable,n_lable)).to(dtype=torch.float32)
    loss = loss_fn(score,lables)
    loss.backward()

    #optimizer.zero_grad()
    optimizer.step()
    return loss

In [ ]:
train_context = torch.tensor(context).to(device=device)
train_target = torch.tensor(target).to(device = device)

# print(train_context.shape)
# print(train_target.shape)

# model = My_CBOW(vocab_size,hidden_size)
# optimizer = torch.optim.Adam(model.parameters())
# loss_fn = torch.nn.BCEWithLogitsLoss()




# ## 取十条负样本
# #sampler = torch.utils.data.Sampler(data_source)

# smapler = UnigramSampler(corpus,0.75,1)


# print(target.shape)


# batch_context = train_context[:10]
# batch_target = train_target[:10]



# negative = smapler.get_negative_sample(batch_target)
# print(negative.shape)

# negative = torch.from_numpy(negative).squeeze(dim=1)
# ##model.train()

# print(f"batch_target_shape{batch_target.shape}")
# train(model,loss_fn,optimizer,batch_context,batch_target,negative)





In [ ]:
 




print(loop)


model = My_CBOW(vocab_size,hidden_size , window_size=window_size)
model = model.to(device=device)

loss_fn = torch.nn.BCEWithLogitsLoss()




                

9295


In [ ]:
batch_size = 100
loop = train_target.shape[0] // batch_size


 
smapler = UnigramSampler(corpus,0.75,5)


loss = 0.0
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

model.train()

In [ ]:



print(f"model weight : {model.inEmbedding.weight}")



for epoch in range(10):
    loss = 0.0
    for i in range(loop):
        start = i * batch_size
        end = (i + 1) * batch_size

        b_context = train_context[start : end]
        b_target =  train_target[start : end]

        negative = smapler.get_negative_sample(b_target)

        negatives = torch.from_numpy(negative.T).to(device= device)

        for j in range(5):
          negative = negatives[j]
          loss += train(model,loss_fn,optimizer,b_context,b_target,negative)


        # negative = smapler.get_negative_sample(b_target)
        # negative = torch.from_numpy(negative).squeeze(dim=1).to(device= device)





      
        # loss += train(model,loss_fn,optimizer,b_context,b_target,negative)

    print(f"{epoch} / {max_epoch} : losss is {loss/batch_size}")

print(f"model weight : {model.inEmbedding.weight}")

model weight : Parameter containing:
tensor([[-1.2032, -1.2957, -1.0738,  ..., -2.4929, -1.1856,  0.7025],
        [ 1.5415, -0.2944,  0.1391,  ..., -1.0367,  1.2740,  0.8600],
        [-0.0066,  3.2815, -1.9544,  ...,  1.4111,  1.0171, -0.4052],
        ...,
        [ 1.3879,  1.0282, -1.6729,  ...,  0.5263,  0.6183, -1.3632],
        [-2.2907, -0.7267, -0.4769,  ...,  2.5100,  0.5578,  0.8489],
        [ 0.5662,  0.5856,  1.2439,  ..., -1.2348, -0.8450, -0.2257]],
       device='cuda:0', requires_grad=True)
0 / 10 : losss is 109.37078857421875
1 / 10 : losss is 108.23054504394531
2 / 10 : losss is 107.28948974609375
3 / 10 : losss is 106.69515991210938


In [ ]:
from common.util import most_similar


## CBOW 模型的评价

print(model.inEmbedding.weight.shape)


word_vecs = model.inEmbedding.weight.detach().to(device = "cpu").numpy()

querys = ['you','year','car','toyota']

for query in querys:
  most_similar(query,word_to_id,id_to_word,word_vecs)




torch.Size([10000, 100])

[query] you
 we: 0.6175280809402466
 i: 0.603376030921936
 they: 0.5740002393722534
 fingers: 0.35948315262794495
 investors: 0.3581792116165161

[query] year
 month: 0.8081380724906921
 week: 0.7605192065238953
 spring: 0.5713254809379578
 summer: 0.5119264721870422
 day: 0.47378748655319214

[query] car
 programming: 0.3897450566291809
 auto: 0.384994775056839
 explosion: 0.37117552757263184
 desire: 0.36241674423217773
 guy: 0.3510245978832245

[query] toyota
 mazda: 0.41037535667419434
 restructured: 0.3895409405231476
 fiat: 0.3817201256752014
 international: 0.3804283142089844
 redmond: 0.35148000717163086
